# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 3 - Introduction aux modèles linéaires généralisés

Ce calepin Jupyter accompage le Chapitre 3 des notes de cours.

Chargeons d'abord les librairies nécessaires.

In [ ]:
using CSV, DataFrames             # Pour charger et organiser les données
using Gadfly                      # Pour générer des graphiques
using Distributions               # Pour utiliser les distributions statistiques
using LinearAlgebra               # Pour utiliser les fonctions d'algèbre linéaire
using GLM                         # Libriairie pour la régression et les modèles linéaires généralisés

Afin de ne pas alourdir le calepin, un fichier de fonctions relatives à la courbe ROC est fourni. Le fichier *functions.jl* contient les fonctions suivantes :
- roc_instance ;
- roc_curve ;
- roc_area.

Il doit être inclus à l'aide de la commande `include("functions.jl")`. Pour avoir une description de ces fonctions, par exemple la fonction *roc_curve*, vous pouvez utiliser la commande `?roc_curve` dans une cellule de code vide.

In [ ]:
include("functions.jl");

In [ ]:
?roc_curve

# Description du jeu de données : les passagers du Titanic

Lors du naufrage du Titanic, entre 1 490 et 1 520 personnes disparaissent sur les 1316 passagers et 889 membres d'équipage. Ce jeu de données compile plusieurs caractéristiques sur chacun des passagers, tels leur âge, leur sexe, la classe de leur billet, etc. ainsi que sa survie ou son décès. Voici toutes les caractéristiques enregistrées ainsi que leurs abréviations :

<ul>
<li>survival : 0 signifie que le passager est décédé, 1 signifie que le passager a survécu.</li>
<li>pclass : la classe du passager. La première classe est la plus dispendieuse.</li>
<li>sex : le genre du passager.</li>
<li>age : age en années. Si le passager a moins d'un an, l'âge peut être donné en fraction. Si l'age est estimé car inconnu, il est donné sous forme d'un nombre à virgule.</li>
<li>sibsp : nombre de frères et soeurs et d'époux du passager voyageant avec lui.</li>
<li>parch : nombre de parents ou d'enfants du passager voyageant avec lui. Des enfants ont voyagé avec une nanny, donc l'enfant avant 0 parent à bord.</li>
<li>ticket : numéro du billet.</li>
<li>fare : tarif.</li>
<li>cabin : numéro de la cabine.</li>
<li>embarked : Port d'embarquement (C = Cherbourg, Q = Queenstown, S = Southampton).</li>
</ul>


Dans ce chapitre et dans le TD3, nous étudierons si certaines caractéristiques ont une influencent sur la probabilité de survie des passagers.

# Chargement des données

Le jeu de données est partionné en deux sous-ensembles : un ensemble d'entraînement et un ensemble de test. L'ensemble d'entraînement, comportant les informations de 872 passagers, sert à estimer les paramètres de votre modèle. La survie ou le décès des 437 passagers de l'ensemble de test est conservé secret pour le concours de prédiction du TD3.

Attention, plusieurs variables comportent des valeurs manquantes.

In [ ]:
# Chargement l'ensemble d'entraînement

data = CSV.read("Train.csv")
first(data,5)

# Analyse exploratoire partielle

Dans le film Titanic, les passagers de première classe semblait avoir une meilleure chance de survie. Vérifions empiriquement (avant d'estimer les paramètres d'un modèle) si cette situation est cohérente avec les données.

In [ ]:
# Affichage du taux de survie des passagers en fonction de leur classe

df = by(data, :Pclass, df -> mean(df[:,:Survived]))
rename!(df, :Pclass => :Classe, :x1 => :Taux_Survie)
plot(df, x=:Classe, y=:Taux_Survie, Geom.bar)

# 3.2 Régression logistique

On utilise seulement la classe comme variable explicative dans le modèle linéaire généralisé.

In [ ]:
# Encoder la classe des passagers avec deux variables indicatrices

Pclass = convert(Array{Int64},data[:,:Pclass])
n = length(Pclass)
x₁ = zeros(Int64,n)
x₂ = zeros(Int64,n)
for i = 1:n
    if Pclass[i]==2
        x₂[i] = 1
    elseif Pclass[i]==1
        x₁[i] = 1
    end   
end

y = data[:,:Survived]

X = hcat(ones(n), x₁, x₂)

# Stockage des variables du modèle dans un dataframe pour une utilisation simple de la librairie GLM
df = DataFrame(x₁=x₁, x₂=x₂, Y=y)
first(df, 5)

## Estimation par maximum de la vraisemblance du modèle de régression logistique

1. La classe des passagers est utilisée comme variable explicative
2. La fonction de lien logit est utilisée

In [ ]:
M = glm(@formula(Y ~ x₁ + x₂), df,  Bernoulli(), LogitLink())

#= Avec les valeurs p affichées, on constate que la classe possède un pouvoir prédictif significatif
pour prédire la survie des passagers. =#

In [ ]:
# Estimation de la probabilité de survie par classe

β̂ = coef(M)

x₀ = [1.0 1.0 0.0 ;
      1.0 0.0 1.0 ; 
      1.0 0.0 0.0]

θ̂₀ = predict(M,x₀)

In [ ]:
# Pour chacune des observations, estimation de la probabilité de survie

θ̂ = predict(M,X)

# 3.2.3 Mesure de la qualité du modèle

In [ ]:
# Calcul du taux de vrais positifs et de faux positifs pour un seuil de 1/2.

p, q = roc_instance(y,θ̂)

println("La sensibilité est de $p.")
println("La spécificité est de $(1-q).")

In [ ]:
# Calcul de la courbe ROC

p, q = roc_curve(y,θ̂)

plot(x=q, y=p, Geom.line, Geom.abline(color="red"), Guide.xlabel("Proportion de faux positifs"),
    Guide.ylabel("Proportion de vrais positifs"))

In [ ]:
# Calcul de l'aire sous la courbe ROC

A = roc_area(y,θ̂)